https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [75]:
import glob
import pandas as pd
import numpy as np
import json

In [76]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [77]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [78]:
annot=pd.concat(annotations_list, ignore_index=True)

In [79]:
annot.columns = ['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [80]:
annot['dict'] = annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [81]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [82]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [83]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [84]:
d=list(zip(text, category, subcat, label))

In [85]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [86]:
data.head()

,text,category,subcategory,label
0,"Effective Date: May 7, 2015 Kraft Site Privacy...",Other,Other Type,Introductory/Generic
1,noSelectedText,First Party Collection/Use,Collection Mode,not-selected
2,collec,First Party Collection/Use,Choice Scope,Collection
3,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
4,"personally-identifiable information, such as",First Party Collection/Use,Personal Information Type,Generic personal information


In [87]:
data['cat_sub'] = data['category'] +'-'+ data['subcategory']

In [91]:
data.groupby('cat_sub').count().to_csv("subcategory_models.csv")


In [15]:
data.groupby(['category','subcategory']).nunique().count()

text           36
category       36
subcategory    36
label          36
cat_sub        36
dtype: int64

In [16]:
data['cat_sub_lab'] = data['category'] +'-'+ data['subcategory'] +'-'+ data['label']

In [17]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data.drop_duplicates()

In [18]:
ds.shape

(36717, 6)

In [19]:
ds.groupby('category').count()

,text,subcategory,label,cat_sub,cat_sub_lab
category,,,,,
Data Retention,676,676,676,676,676
Data Security,808,808,808,808,808
Do Not Track,67,67,67,67,67
First Party Collection/Use,15346,15346,15346,15346,15346
International and Specific Audiences,582,582,582,582,582
Other,2759,2759,2759,2759,2759
Policy Change,932,932,932,932,932
Third Party Sharing/Collection,10655,10655,10655,10655,10655
"User Access, Edit and Deletion",1094,1094,1094,1094,1094


In [20]:
ds.groupby(['category','subcategory']).count().shape

(36, 4)

In [21]:
ds.groupby(['category','subcategory','label']).count().shape

(259, 3)

In [22]:
ds['tokenized'] = ds['text'].apply(lambda x : re.split(' ', x))
# ds['tokenized_text']=res

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
w2v = Word2Vec.load('word2vec.model')

2019-09-19 21:37:46,412 : INFO : loading Word2Vec object from word2vec.model
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-09-19 21:37:46,460 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2019-09-19 21:37:46,462 : INFO : setting ignored attribute vectors_norm to None
2019-09-19 21:37:46,464 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2019-09-19 21:37:46,465 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2019-09-19 21:37:46,469 : INFO : setting ignored attribute cum_table to None
2019-09-19 21:37:46,470 : INFO : loaded word2vec.model


In [25]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()

In [26]:
ds.reset_index(inplace=True, drop=True)

In [27]:
word_vectors = w2v.wv

In [28]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [29]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [30]:
len(word2id)

3933

In [31]:
embedding_matrix.shape

(3933, 100)

In [32]:
embedding_matrix[0].shape

(100,)

In [33]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [34]:
len(embedding_matrix)

3934

In [35]:
embedding_matrix[3933].shape

(100,)

In [36]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(ds.tokenized, word2id)
x.shape

Data created. Percentage of unknown words: 7006.000


(36717,)

In [37]:
ds['enumerated_text']=x

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
data_retention = ds[ds.category=='Data Retention']
data_retention['Personal_Information_Type'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
data_retention['Retention_Period'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Retention Period' else 0)
data_retention['Retention_Purpose'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Retention Purpose' else 0)

first_party_collection_use = ds[ds.category=='First Party Collection/Use']
first_party_collection_use['Action_First_Party'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Action First-Party' else 0)
first_party_collection_use['Choice_Scope'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
first_party_collection_use['Choice_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
first_party_collection_use['Collection_Mode'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Collection Mode' else 0)
first_party_collection_use['Does_Does_Not'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Does/Does Not' else 0)
first_party_collection_use['Identifiability'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Identifiability' else 0)
first_party_collection_use['Personal_Information_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
first_party_collection_use['Purpose'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
first_party_collection_use['User_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

policy_change = ds[ds.category=='Policy Change']
policy_change['Change_Type'] = policy_change['subcategory'].apply(lambda x: 1 if x=='Change Type' else 0)
policy_change['Notification_Type'] = policy_change['subcategory'].apply(lambda x: 1 if x=='Notification Type' else 0)
policy_change['User_Choice'] = policy_change['subcategory'].apply(lambda x: 1 if x=='User Choice' else 0)

third_party_sharing_collection = ds[ds.category=='Third Party Sharing/Collection']
third_party_sharing_collection['Action_Third_Party'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Action Third Party' else 0)
third_party_sharing_collection['Choice_Scope'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
third_party_sharing_collection['Choice_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
third_party_sharing_collection['Does_Does_Not'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Does/Does Not' else 0)
third_party_sharing_collection['Identifiability'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Identifiability' else 0)
third_party_sharing_collection['Personal_Information_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
third_party_sharing_collection['Purpose'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
third_party_sharing_collection['Third_Party_Entity'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Third Party Entity' else 0)
third_party_sharing_collection['User_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

user_access_edit_deletion = ds[ds.category=='User Access, Edit and Deletion']
user_access_edit_deletion['Access_Scope'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='Access Scope' else 0)
user_access_edit_deletion['Access_Type'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='Access Type' else 0)
user_access_edit_deletion['User_Type'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

user_choice_control = ds[ds.category=='User Choice/Control']
user_choice_control['Choice_Scope'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
user_choice_control['Choice_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
user_choice_control['Personal_Information_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
user_choice_control['Purpose'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
user_choice_control['User_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)


/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [39]:
#first_party_collection_use.sum()

In [40]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.model_selection import train_test_split

max_length = max(ds.enumerated_text.apply(lambda x: len(x)))

Using TensorFlow backend.


DATA RETENTION MODELS

In [41]:
# Personal Information Type Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Personal_Information_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 3.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('data_retention_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 473 samples, validate on 203 samples
Epoch 1/5
473/473 [==============================] - 1s 2ms/step - loss: 0.8819 - acc: 0.6512 - val_loss: 0.4620 - val_acc: 0.7537
Epoch 2/5
473/473 [==============================] - 0s 644us/step - loss: 0.4072 - acc: 0.8689 - val_loss: 0.3287 - val_acc: 0.8374
Epoch 3/5
473/473 [==============================] - 0s 676us/step - loss: 0.2757 - acc: 0.9281 - val_loss: 0.3415 - val_acc: 0.8325
Epoch 4/5
473/473 [==============================] - 0s 654us/step - loss: 0.2096 - acc: 0.9577 - val_loss: 0.3402 - val_acc: 0.8374
Epoch 5/5
473/473 [==============================] - 0s 672us/step - loss: 0.1744 - acc: 0.9662 - val_loss: 0.3393 - val_acc: 0.8621
[[333  10]
 [  6 124]]
0.9393939393939394
0.9538461538461539
[[128  17]
 [ 11  47]]
0.7704918032786885
0.8103448275862069


In [42]:
# Retention Period Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Retention_Period, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 2.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('data_retention_rentention_period.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 473 samples, validate on 203 samples
Epoch 1/5
473/473 [==============================] - 1s 2ms/step - loss: 0.8654 - acc: 0.6575 - val_loss: 0.6800 - val_acc: 0.7291
Epoch 2/5
473/473 [==============================] - 0s 673us/step - loss: 0.5727 - acc: 0.8034 - val_loss: 0.6251 - val_acc: 0.7291
Epoch 3/5
473/473 [==============================] - 0s 644us/step - loss: 0.4399 - acc: 0.8710 - val_loss: 0.6615 - val_acc: 0.7340
Epoch 4/5
473/473 [==============================] - 0s 604us/step - loss: 0.3468 - acc: 0.9027 - val_loss: 0.7058 - val_acc: 0.7488
Epoch 5/5
473/473 [==============================] - 0s 695us/step - loss: 0.2873 - acc: 0.9070 - val_loss: 0.7626 - val_acc: 0.7586
[[280  31]
 [  7 155]]
0.8908045977011495
0.9567901234567902
[[103  28]
 [ 21  51]]
0.6754966887417219
0.7083333333333334


In [43]:
# Retention Purpose Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Retention_Purpose, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 2.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('data_retention_retention_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 473 samples, validate on 203 samples
Epoch 1/5
473/473 [==============================] - 1s 2ms/step - loss: 0.9025 - acc: 0.6025 - val_loss: 0.6885 - val_acc: 0.6650
Epoch 2/5
473/473 [==============================] - 0s 626us/step - loss: 0.5029 - acc: 0.8436 - val_loss: 0.7028 - val_acc: 0.7537
Epoch 3/5
473/473 [==============================] - 0s 641us/step - loss: 0.3654 - acc: 0.8901 - val_loss: 0.8175 - val_acc: 0.7340
Epoch 4/5
473/473 [==============================] - 0s 644us/step - loss: 0.2844 - acc: 0.9091 - val_loss: 0.8280 - val_acc: 0.7635
Epoch 5/5
473/473 [==============================] - 0s 621us/step - loss: 0.2336 - acc: 0.9345 - val_loss: 0.8439 - val_acc: 0.7340
[[271  21]
 [  6 175]]
0.9283819628647214
0.9668508287292817
[[101  29]
 [ 25  48]]
0.64
0.6575342465753424


FIRST PARTY COLLECTION/USE

In [44]:
# Action First-Party Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Action_First_Party, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_action_first_party.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 650us/step - loss: 1.0256 - acc: 0.6457 - val_loss: 0.5653 - val_acc: 0.7094
Epoch 2/5
10742/10742 [==============================] - 6s 594us/step - loss: 0.8222 - acc: 0.7594 - val_loss: 0.5620 - val_acc: 0.7233
Epoch 3/5
10742/10742 [==============================] - 6s 592us/step - loss: 0.6994 - acc: 0.8000 - val_loss: 0.5616 - val_acc: 0.7374
Epoch 4/5
10742/10742 [==============================] - 6s 598us/step - loss: 0.5912 - acc: 0.8334 - val_loss: 0.6251 - val_acc: 0.7096
Epoch 5/5
10742/10742 [==============================] - 6s 586us/step - loss: 0.5275 - acc: 0.8529 - val_loss: 0.5692 - val_acc: 0.7483
[[7483 1318]
 [  87 1854]]
0.725210248386466
0.955177743431221
[[2967  845]
 [ 314  478]]
0.45200945626477534
0.6035353535353535


In [45]:
# Choice Scope Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Choice_Scope, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 15.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 657us/step - loss: 0.8632 - acc: 0.8686 - val_loss: 0.4103 - val_acc: 0.7974
Epoch 2/5
10742/10742 [==============================] - 6s 598us/step - loss: 0.6459 - acc: 0.8586 - val_loss: 0.3393 - val_acc: 0.8454
Epoch 3/5
10742/10742 [==============================] - 6s 592us/step - loss: 0.5269 - acc: 0.8886 - val_loss: 0.3600 - val_acc: 0.8273
Epoch 4/5
10742/10742 [==============================] - 6s 602us/step - loss: 0.4202 - acc: 0.9048 - val_loss: 0.2887 - val_acc: 0.8827
Epoch 5/5
10742/10742 [==============================] - 6s 584us/step - loss: 0.3489 - acc: 0.9232 - val_loss: 0.3028 - val_acc: 0.8747
[[9447  946]
 [  19  330]]
0.4061538461538462
0.9455587392550143
[[3954  471]
 [ 106   73]]
0.20193637621023514
0.40782122905027934


In [46]:
# Choice Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Choice_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 646us/step - loss: 0.5110 - acc: 0.8996 - val_loss: 0.2588 - val_acc: 0.9118
Epoch 2/5
10742/10742 [==============================] - 6s 594us/step - loss: 0.3325 - acc: 0.9337 - val_loss: 0.2409 - val_acc: 0.9112
Epoch 3/5
10742/10742 [==============================] - 6s 598us/step - loss: 0.2703 - acc: 0.9423 - val_loss: 0.2236 - val_acc: 0.9209
Epoch 4/5
10742/10742 [==============================] - 6s 605us/step - loss: 0.2238 - acc: 0.9472 - val_loss: 0.2279 - val_acc: 0.9177
Epoch 5/5
10742/10742 [==============================] - 6s 584us/step - loss: 0.1927 - acc: 0.9528 - val_loss: 0.2553 - val_acc: 0.9129
[[9656  457]
 [  40  589]]
0.7032835820895523
0.9364069952305246
[[4028  268]
 [ 133  175]]
0.4660452729693742
0.5681818181818182


In [47]:
# Collection Mode Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Collection_Mode, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_collection_mode.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 9s 793us/step - loss: 0.8486 - acc: 0.8133 - val_loss: 0.4942 - val_acc: 0.8056
Epoch 2/5
10742/10742 [==============================] - 7s 683us/step - loss: 0.7026 - acc: 0.8473 - val_loss: 0.4133 - val_acc: 0.8267
Epoch 3/5
10742/10742 [==============================] - 7s 648us/step - loss: 0.6173 - acc: 0.8553 - val_loss: 0.4526 - val_acc: 0.7991
Epoch 4/5
10742/10742 [==============================] - 7s 641us/step - loss: 0.5452 - acc: 0.8648 - val_loss: 0.4661 - val_acc: 0.7902
Epoch 5/5
10742/10742 [==============================] - 7s 654us/step - loss: 0.4726 - acc: 0.8796 - val_loss: 0.4847 - val_acc: 0.7921
[[8499 1200]
 [  88  955]]
0.5972482801751096
0.9156279961649089
[[3428  691]
 [ 266  219]]
0.31397849462365596
0.4515463917525773


In [48]:
# Does/Does Not Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Does_Does_Not, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_does_does_not.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 683us/step - loss: 0.7260 - acc: 0.8265 - val_loss: 0.4293 - val_acc: 0.8223
Epoch 2/5
10742/10742 [==============================] - 6s 596us/step - loss: 0.5767 - acc: 0.8505 - val_loss: 0.3556 - val_acc: 0.8377
Epoch 3/5
10742/10742 [==============================] - 6s 591us/step - loss: 0.4916 - acc: 0.8744 - val_loss: 0.3670 - val_acc: 0.8354
Epoch 4/5
10742/10742 [==============================] - 6s 596us/step - loss: 0.4270 - acc: 0.8919 - val_loss: 0.3757 - val_acc: 0.8393
Epoch 5/5
10742/10742 [==============================] - 6s 588us/step - loss: 0.3780 - acc: 0.9007 - val_loss: 0.4148 - val_acc: 0.8288
[[8894  927]
 [  67  854]]
0.6321243523316062
0.9272529858849077
[[3629  588]
 [ 200  187]]
0.3218588640275387
0.48320413436692505


In [49]:
# Identifiability Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Identifiability, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_identifiability.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 660us/step - loss: 0.4570 - acc: 0.9385 - val_loss: 0.1740 - val_acc: 0.9553
Epoch 2/5
10742/10742 [==============================] - 6s 603us/step - loss: 0.3161 - acc: 0.9433 - val_loss: 0.1883 - val_acc: 0.9429
Epoch 3/5
10742/10742 [==============================] - 6s 596us/step - loss: 0.2433 - acc: 0.9546 - val_loss: 0.1849 - val_acc: 0.9407
Epoch 4/5
10742/10742 [==============================] - 6s 598us/step - loss: 0.1907 - acc: 0.9609 - val_loss: 0.1965 - val_acc: 0.9340
Epoch 5/5
10742/10742 [==============================] - 6s 597us/step - loss: 0.1567 - acc: 0.9653 - val_loss: 0.1752 - val_acc: 0.9513
[[10098   189]
 [   68   387]]
0.7507274490785645
0.8505494505494505
[[4301  122]
 [ 102   79]]
0.41361256544502617
0.43646408839779005


In [50]:
# Personal Information Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Personal_Information_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 659us/step - loss: 0.8838 - acc: 0.7410 - val_loss: 0.4987 - val_acc: 0.7778
Epoch 2/5
10742/10742 [==============================] - 6s 597us/step - loss: 0.6448 - acc: 0.8237 - val_loss: 0.4066 - val_acc: 0.8308
Epoch 3/5
10742/10742 [==============================] - 7s 676us/step - loss: 0.5228 - acc: 0.8613 - val_loss: 0.4564 - val_acc: 0.8141
Epoch 4/5
10742/10742 [==============================] - 10s 944us/step - loss: 0.4278 - acc: 0.8896 - val_loss: 0.4471 - val_acc: 0.8230
Epoch 5/5
10742/10742 [==============================] - 7s 626us/step - loss: 0.3684 - acc: 0.9049 - val_loss: 0.4214 - val_acc: 0.8486
[[7727  788]
 [  52 2175]]
0.838150289017341
0.9766502020655591
[[3195  517]
 [ 180  712]]
0.6713814238566713
0.7982062780269058


In [51]:
# Purpose Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Purpose, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 8s 699us/step - loss: 0.8864 - acc: 0.7306 - val_loss: 0.4752 - val_acc: 0.7887
Epoch 2/5
10742/10742 [==============================] - 6s 601us/step - loss: 0.6514 - acc: 0.8264 - val_loss: 0.5201 - val_acc: 0.7639
Epoch 3/5
10742/10742 [==============================] - 6s 595us/step - loss: 0.5296 - acc: 0.8673 - val_loss: 0.4734 - val_acc: 0.8093
Epoch 4/5
10742/10742 [==============================] - 6s 605us/step - loss: 0.4346 - acc: 0.8953 - val_loss: 0.4235 - val_acc: 0.8380
Epoch 5/5
10742/10742 [==============================] - 6s 592us/step - loss: 0.3607 - acc: 0.9172 - val_loss: 0.4223 - val_acc: 0.8506
[[7177  627]
 [  65 2873]]
0.8925132028580304
0.9778761061946902
[[2867  461]
 [ 227 1049]]
0.7530509691313712
0.8221003134796239


In [52]:
# User Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.User_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 10742 samples, validate on 4604 samples
Epoch 1/5
10742/10742 [==============================] - 7s 690us/step - loss: 0.2934 - acc: 0.9696 - val_loss: 0.1913 - val_acc: 0.9290
Epoch 2/5
10742/10742 [==============================] - 8s 765us/step - loss: 0.1809 - acc: 0.9714 - val_loss: 0.1189 - val_acc: 0.9639
Epoch 3/5
10742/10742 [==============================] - 8s 740us/step - loss: 0.1362 - acc: 0.9758 - val_loss: 0.0921 - val_acc: 0.9718
Epoch 4/5
10742/10742 [==============================] - 7s 611us/step - loss: 0.1104 - acc: 0.9791 - val_loss: 0.1157 - val_acc: 0.9611
Epoch 5/5
10742/10742 [==============================] - 6s 604us/step - loss: 0.0857 - acc: 0.9837 - val_loss: 0.1061 - val_acc: 0.9687
[[10393   110]
 [   21   218]]
0.7689594356261024
0.9121338912133892
[[4411   89]
 [  55   49]]
0.4049586776859504
0.47115384615384615


POLICY CHANGE

In [53]:
# Change Type Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.Change_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('policy_change_change_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 652 samples, validate on 280 samples
Epoch 1/5
652/652 [==============================] - 1s 2ms/step - loss: 1.4465 - acc: 0.4294 - val_loss: 0.7795 - val_acc: 0.5929
Epoch 2/5
652/652 [==============================] - 0s 615us/step - loss: 0.9534 - acc: 0.6994 - val_loss: 0.8716 - val_acc: 0.6214
Epoch 3/5
652/652 [==============================] - 0s 690us/step - loss: 0.7319 - acc: 0.7423 - val_loss: 0.7400 - val_acc: 0.6786
Epoch 4/5
652/652 [==============================] - 0s 656us/step - loss: 0.6026 - acc: 0.8206 - val_loss: 0.8499 - val_acc: 0.6536
Epoch 5/5
652/652 [==============================] - 0s 664us/step - loss: 0.5276 - acc: 0.8252 - val_loss: 0.8164 - val_acc: 0.6893
[[317  91]
 [  3 241]]
0.8368055555555556
0.9877049180327869
[[121  62]
 [ 25  72]]
0.6233766233766234
0.7422680412371134


In [54]:
# Notification Type Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.Notification_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('policy_change_notification_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 652 samples, validate on 280 samples
Epoch 1/5
652/652 [==============================] - 1s 2ms/step - loss: 1.4417 - acc: 0.5015 - val_loss: 0.9353 - val_acc: 0.5536
Epoch 2/5
652/652 [==============================] - 1s 971us/step - loss: 0.9332 - acc: 0.6672 - val_loss: 0.7854 - val_acc: 0.6107
Epoch 3/5
652/652 [==============================] - 1s 792us/step - loss: 0.7301 - acc: 0.7193 - val_loss: 0.8323 - val_acc: 0.6143
Epoch 4/5
652/652 [==============================] - 1s 778us/step - loss: 0.6457 - acc: 0.7684 - val_loss: 0.7942 - val_acc: 0.6821
Epoch 5/5
652/652 [==============================] - 1s 774us/step - loss: 0.5942 - acc: 0.7945 - val_loss: 0.8661 - val_acc: 0.6857
[[272 114]
 [  3 263]]
0.818040435458787
0.9887218045112782
[[ 77  66]
 [ 22 115]]
0.7232704402515723
0.8394160583941606


In [55]:
# User Choice Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.User_Choice, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('policy_change_user_choice.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 652 samples, validate on 280 samples
Epoch 1/5
652/652 [==============================] - 1s 2ms/step - loss: 1.2145 - acc: 0.5107 - val_loss: 0.6357 - val_acc: 0.7286
Epoch 2/5
652/652 [==============================] - 0s 625us/step - loss: 0.7707 - acc: 0.8528 - val_loss: 0.6976 - val_acc: 0.6536
Epoch 3/5
652/652 [==============================] - 0s 655us/step - loss: 0.6176 - acc: 0.8236 - val_loss: 0.6321 - val_acc: 0.7286
Epoch 4/5
652/652 [==============================] - 0s 641us/step - loss: 0.5031 - acc: 0.8666 - val_loss: 0.6787 - val_acc: 0.7179
Epoch 5/5
652/652 [==============================] - 0s 667us/step - loss: 0.4478 - acc: 0.8773 - val_loss: 0.6887 - val_acc: 0.7214
[[439  71]
 [  4 138]]
0.7863247863247862
0.971830985915493
[[178  56]
 [ 22  24]]
0.38095238095238093
0.5217391304347826


THIRD PARTY SHARING/COLLECTION

In [57]:
# Action Third-Party Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Action_Third_Party, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_action_third_party.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 6s 797us/step - loss: 1.0042 - acc: 0.6881 - val_loss: 0.5508 - val_acc: 0.7366
Epoch 2/5
7458/7458 [==============================] - 5s 637us/step - loss: 0.7731 - acc: 0.7776 - val_loss: 0.5128 - val_acc: 0.7557
Epoch 3/5
7458/7458 [==============================] - 6s 822us/step - loss: 0.6564 - acc: 0.8159 - val_loss: 0.5133 - val_acc: 0.7660
Epoch 4/5
7458/7458 [==============================] - 8s 1ms/step - loss: 0.5600 - acc: 0.8459 - val_loss: 0.5531 - val_acc: 0.7657
Epoch 5/5
7458/7458 [==============================] - 6s 748us/step - loss: 0.4931 - acc: 0.8615 - val_loss: 0.5451 - val_acc: 0.7710
[[5309  841]
 [  40 1268]]
0.7421714954638572
0.9694189602446484
[[2087  529]
 [ 203  378]]
0.5080645161290323
0.6506024096385542


In [58]:
# Choice Scope Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Choice_Scope, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 5s 731us/step - loss: 0.4768 - acc: 0.9553 - val_loss: 0.2159 - val_acc: 0.9687
Epoch 2/5
7458/7458 [==============================] - 4s 599us/step - loss: 0.3349 - acc: 0.9623 - val_loss: 0.1469 - val_acc: 0.9653
Epoch 3/5
7458/7458 [==============================] - 4s 592us/step - loss: 0.2742 - acc: 0.9627 - val_loss: 0.1723 - val_acc: 0.9556
Epoch 4/5
7458/7458 [==============================] - 4s 595us/step - loss: 0.2335 - acc: 0.9603 - val_loss: 0.1735 - val_acc: 0.9503
Epoch 5/5
7458/7458 [==============================] - 4s 595us/step - loss: 0.1968 - acc: 0.9646 - val_loss: 0.1743 - val_acc: 0.9446
[[7032  183]
 [  58  185]]
0.6055646481178396
0.7613168724279835
[[2994  104]
 [  73   26]]
0.22707423580786026
0.26262626262626265


In [59]:
# Choice Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Choice_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 6s 743us/step - loss: 0.4627 - acc: 0.9128 - val_loss: 0.2735 - val_acc: 0.8993
Epoch 2/5
7458/7458 [==============================] - 5s 605us/step - loss: 0.2513 - acc: 0.9501 - val_loss: 0.1800 - val_acc: 0.9396
Epoch 3/5
7458/7458 [==============================] - 5s 616us/step - loss: 0.1863 - acc: 0.9602 - val_loss: 0.1816 - val_acc: 0.9396
Epoch 4/5
7458/7458 [==============================] - 5s 609us/step - loss: 0.1469 - acc: 0.9702 - val_loss: 0.1574 - val_acc: 0.9506
Epoch 5/5
7458/7458 [==============================] - 4s 590us/step - loss: 0.1169 - acc: 0.9772 - val_loss: 0.1697 - val_acc: 0.9468
[[6864  125]
 [  11  458]]
0.870722433460076
0.976545842217484
[[2883  102]
 [  68  144]]
0.62882096069869
0.6792452830188679


In [60]:
# Does/Does Not Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Does_Does_Not, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_does_does_not.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 6s 780us/step - loss: 0.7579 - acc: 0.8097 - val_loss: 0.3520 - val_acc: 0.8233
Epoch 2/5
7458/7458 [==============================] - 4s 602us/step - loss: 0.5698 - acc: 0.8459 - val_loss: 0.4073 - val_acc: 0.8123
Epoch 3/5
7458/7458 [==============================] - 5s 607us/step - loss: 0.5071 - acc: 0.8591 - val_loss: 0.4047 - val_acc: 0.8086
Epoch 4/5
7458/7458 [==============================] - 4s 595us/step - loss: 0.4625 - acc: 0.8718 - val_loss: 0.4175 - val_acc: 0.8151
Epoch 5/5
7458/7458 [==============================] - 4s 600us/step - loss: 0.4200 - acc: 0.8837 - val_loss: 0.4346 - val_acc: 0.8076
[[6023  776]
 [  51  608]]
0.5952031326480667
0.9226100151745068
[[2467  452]
 [ 163  115]]
0.2721893491124261
0.4136690647482014


In [61]:
# Identifiability Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Identifiability, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_identifiability.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 6s 786us/step - loss: 0.4198 - acc: 0.9358 - val_loss: 0.2085 - val_acc: 0.9309
Epoch 2/5
7458/7458 [==============================] - 5s 630us/step - loss: 0.2539 - acc: 0.9520 - val_loss: 0.1564 - val_acc: 0.9431
Epoch 3/5
7458/7458 [==============================] - 5s 736us/step - loss: 0.1968 - acc: 0.9566 - val_loss: 0.1496 - val_acc: 0.9478
Epoch 4/5
7458/7458 [==============================] - 7s 888us/step - loss: 0.1675 - acc: 0.9615 - val_loss: 0.1528 - val_acc: 0.9443
Epoch 5/5
7458/7458 [==============================] - 6s 853us/step - loss: 0.1416 - acc: 0.9643 - val_loss: 0.1480 - val_acc: 0.9475
[[6990  194]
 [  34  240]]
0.6779661016949152
0.8759124087591241
[[2975  110]
 [  58   54]]
0.391304347826087
0.48214285714285715


In [62]:
# Personal Information Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Personal_Information_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 5s 711us/step - loss: 0.6842 - acc: 0.8384 - val_loss: 0.2926 - val_acc: 0.8883
Epoch 2/5
7458/7458 [==============================] - 6s 793us/step - loss: 0.4653 - acc: 0.8914 - val_loss: 0.2960 - val_acc: 0.8858
Epoch 3/5
7458/7458 [==============================] - 6s 843us/step - loss: 0.3854 - acc: 0.9091 - val_loss: 0.2731 - val_acc: 0.9015
Epoch 4/5
7458/7458 [==============================] - 6s 795us/step - loss: 0.3212 - acc: 0.9265 - val_loss: 0.3020 - val_acc: 0.8861
Epoch 5/5
7458/7458 [==============================] - 5s 722us/step - loss: 0.2754 - acc: 0.9370 - val_loss: 0.2843 - val_acc: 0.8965
[[6003  425]
 [  26 1004]]
0.8165921106140707
0.974757281553398
[[2504  260]
 [  71  362]]
0.6862559241706161
0.836027713625866


In [63]:
# Purpose Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Purpose, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 7s 905us/step - loss: 0.9501 - acc: 0.6984 - val_loss: 0.5439 - val_acc: 0.7554
Epoch 2/5
7458/7458 [==============================] - 7s 899us/step - loss: 0.6611 - acc: 0.8222 - val_loss: 0.5203 - val_acc: 0.7764
Epoch 3/5
7458/7458 [==============================] - 5s 680us/step - loss: 0.5516 - acc: 0.8584 - val_loss: 0.5019 - val_acc: 0.7967
Epoch 4/5
7458/7458 [==============================] - 4s 499us/step - loss: 0.4731 - acc: 0.8851 - val_loss: 0.4654 - val_acc: 0.8308
Epoch 5/5
7458/7458 [==============================] - 4s 505us/step - loss: 0.3970 - acc: 0.9091 - val_loss: 0.5056 - val_acc: 0.8214
[[5017  521]
 [  51 1869]]
0.8672853828306264
0.9734375
[[1969  421]
 [ 150  657]]
0.6970822281167108
0.8141263940520446


In [64]:
# Third Part Entity Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Third_Party_Entity, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_third_party_entity.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 8s 1ms/step - loss: 0.9144 - acc: 0.7247 - val_loss: 0.4618 - val_acc: 0.7839
Epoch 2/5
7458/7458 [==============================] - 4s 544us/step - loss: 0.6725 - acc: 0.8215 - val_loss: 0.4633 - val_acc: 0.8048
Epoch 3/5
7458/7458 [==============================] - 4s 528us/step - loss: 0.5583 - acc: 0.8663 - val_loss: 0.4409 - val_acc: 0.8133
Epoch 4/5
7458/7458 [==============================] - 7s 881us/step - loss: 0.4595 - acc: 0.8903 - val_loss: 0.4500 - val_acc: 0.8273
Epoch 5/5
7458/7458 [==============================] - 5s 680us/step - loss: 0.4044 - acc: 0.9084 - val_loss: 0.4633 - val_acc: 0.8286
[[5451  553]
 [  42 1412]]
0.8259725065808715
0.9711141678129298
[[2181  376]
 [ 172  468]]
0.6307277628032345
0.73125


In [65]:
# User Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.User_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 7458 samples, validate on 3197 samples
Epoch 1/5
7458/7458 [==============================] - 8s 1ms/step - loss: 0.2296 - acc: 0.9796 - val_loss: 0.0982 - val_acc: 0.9875
Epoch 2/5
7458/7458 [==============================] - 6s 829us/step - loss: 0.1206 - acc: 0.9851 - val_loss: 0.0553 - val_acc: 0.9881
Epoch 3/5
7458/7458 [==============================] - 5s 649us/step - loss: 0.0823 - acc: 0.9890 - val_loss: 0.0573 - val_acc: 0.9847
Epoch 4/5
7458/7458 [==============================] - 5s 644us/step - loss: 0.0603 - acc: 0.9905 - val_loss: 0.0570 - val_acc: 0.9853
Epoch 5/5
7458/7458 [==============================] - 5s 611us/step - loss: 0.0463 - acc: 0.9928 - val_loss: 0.0532 - val_acc: 0.9869
[[7313   44]
 [  10   91]]
0.7711864406779662
0.900990099009901
[[3133   29]
 [  13   22]]
0.5116279069767442
0.6285714285714286


USER ACCESS, EDIT, AND DELETION

In [66]:
# Access Scope Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.Access_Scope, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_access_edit_deletion_access_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 765 samples, validate on 329 samples
Epoch 1/5
765/765 [==============================] - 2s 2ms/step - loss: 1.4039 - acc: 0.4183 - val_loss: 0.7786 - val_acc: 0.4711
Epoch 2/5
765/765 [==============================] - 0s 618us/step - loss: 0.9382 - acc: 0.6261 - val_loss: 0.7972 - val_acc: 0.5836
Epoch 3/5
765/765 [==============================] - 0s 622us/step - loss: 0.7607 - acc: 0.7712 - val_loss: 0.7491 - val_acc: 0.6930
Epoch 4/5
765/765 [==============================] - 0s 625us/step - loss: 0.6562 - acc: 0.8078 - val_loss: 0.8302 - val_acc: 0.6565
Epoch 5/5
765/765 [==============================] - 0s 627us/step - loss: 0.5547 - acc: 0.8588 - val_loss: 0.8303 - val_acc: 0.6960
[[373  98]
 [  6 288]]
0.8470588235294118
0.9795918367346939
[[134  68]
 [ 32  95]]
0.6551724137931034
0.7480314960629921


In [67]:
# Access Type Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.Access_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_access_edit_deletion_access_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 765 samples, validate on 329 samples
Epoch 1/5
765/765 [==============================] - 2s 2ms/step - loss: 1.5148 - acc: 0.4850 - val_loss: 0.8844 - val_acc: 0.5076
Epoch 2/5
765/765 [==============================] - 0s 629us/step - loss: 0.9756 - acc: 0.6183 - val_loss: 1.0319 - val_acc: 0.5562
Epoch 3/5
765/765 [==============================] - 1s 656us/step - loss: 0.7977 - acc: 0.7294 - val_loss: 0.8235 - val_acc: 0.6991
Epoch 4/5
765/765 [==============================] - 0s 619us/step - loss: 0.6379 - acc: 0.7791 - val_loss: 0.8267 - val_acc: 0.7082
Epoch 5/5
765/765 [==============================] - 0s 615us/step - loss: 0.5530 - acc: 0.8405 - val_loss: 0.9402 - val_acc: 0.6960
[[273 129]
 [  3 360]]
0.8450704225352113
0.9917355371900827
[[100  78]
 [ 22 129]]
0.7206703910614525
0.8543046357615894


In [68]:
# User Type Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.User_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_access_edit_deletion_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 765 samples, validate on 329 samples
Epoch 1/5
765/765 [==============================] - 2s 2ms/step - loss: 0.9084 - acc: 0.6732 - val_loss: 0.4542 - val_acc: 0.8450
Epoch 2/5
765/765 [==============================] - 0s 621us/step - loss: 0.5366 - acc: 0.8706 - val_loss: 0.4711 - val_acc: 0.8419
Epoch 3/5
765/765 [==============================] - 0s 629us/step - loss: 0.4201 - acc: 0.8627 - val_loss: 0.4509 - val_acc: 0.8663
Epoch 4/5
765/765 [==============================] - 0s 622us/step - loss: 0.3544 - acc: 0.9124 - val_loss: 0.4830 - val_acc: 0.8663
Epoch 5/5
765/765 [==============================] - 0s 653us/step - loss: 0.3071 - acc: 0.8967 - val_loss: 0.4772 - val_acc: 0.8845
[[612  45]
 [  7 101]]
0.795275590551181
0.9351851851851852
[[252  26]
 [ 12  39]]
0.6724137931034482
0.7647058823529411


USER CHOICE/CONTROL

In [70]:
# Choice Scope Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Choice_Scope, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 2658 samples, validate on 1140 samples
Epoch 1/5
2658/2658 [==============================] - 3s 1ms/step - loss: 1.3325 - acc: 0.4108 - val_loss: 0.8313 - val_acc: 0.4684
Epoch 2/5
2658/2658 [==============================] - 2s 654us/step - loss: 1.0718 - acc: 0.6065 - val_loss: 0.8020 - val_acc: 0.5035
Epoch 3/5
2658/2658 [==============================] - 2s 627us/step - loss: 0.9292 - acc: 0.6697 - val_loss: 0.8092 - val_acc: 0.5596
Epoch 4/5
2658/2658 [==============================] - 2s 630us/step - loss: 0.8159 - acc: 0.7378 - val_loss: 0.7550 - val_acc: 0.6228
Epoch 5/5
2658/2658 [==============================] - 2s 698us/step - loss: 0.7079 - acc: 0.7904 - val_loss: 0.7698 - val_acc: 0.6368
[[1535  368]
 [  39  716]]
0.77868406742795
0.9483443708609272
[[504 292]
 [122 222]]
0.5174825174825175
0.6453488372093024


In [71]:
# Choice Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Choice_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 2658 samples, validate on 1140 samples
Epoch 1/5
2658/2658 [==============================] - 4s 2ms/step - loss: 1.1361 - acc: 0.6166 - val_loss: 0.6711 - val_acc: 0.6956
Epoch 2/5
2658/2658 [==============================] - 2s 791us/step - loss: 0.7587 - acc: 0.7784 - val_loss: 0.6881 - val_acc: 0.7096
Epoch 3/5
2658/2658 [==============================] - 2s 643us/step - loss: 0.6166 - acc: 0.8198 - val_loss: 0.6226 - val_acc: 0.7482
Epoch 4/5
2658/2658 [==============================] - 2s 634us/step - loss: 0.5241 - acc: 0.8563 - val_loss: 0.6342 - val_acc: 0.7518
Epoch 5/5
2658/2658 [==============================] - 2s 628us/step - loss: 0.4542 - acc: 0.8864 - val_loss: 0.6448 - val_acc: 0.7693
[[1536  227]
 [  17  878]]
0.8779999999999999
0.9810055865921787
[[580 193]
 [ 70 297]]
0.6931155192532089
0.8092643051771117


In [72]:
# Personal Information Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Personal_Information_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 2658 samples, validate on 1140 samples
Epoch 1/5
2658/2658 [==============================] - 3s 1ms/step - loss: 0.8621 - acc: 0.7761 - val_loss: 0.4588 - val_acc: 0.8070
Epoch 2/5
2658/2658 [==============================] - 2s 660us/step - loss: 0.5762 - acc: 0.8683 - val_loss: 0.4348 - val_acc: 0.8114
Epoch 3/5
2658/2658 [==============================] - 2s 668us/step - loss: 0.4708 - acc: 0.8837 - val_loss: 0.4027 - val_acc: 0.8377
Epoch 4/5
2658/2658 [==============================] - 2s 675us/step - loss: 0.4043 - acc: 0.8984 - val_loss: 0.4109 - val_acc: 0.8386
Epoch 5/5
2658/2658 [==============================] - 2s 693us/step - loss: 0.3537 - acc: 0.9108 - val_loss: 0.4272 - val_acc: 0.8404
[[2115  201]
 [  26  316]]
0.7357392316647263
0.9239766081871345
[[872 118]
 [ 64  86]]
0.48587570621468934
0.5733333333333334


In [73]:
# Purpose Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Purpose, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 2658 samples, validate on 1140 samples
Epoch 1/5
2658/2658 [==============================] - 3s 1ms/step - loss: 1.1136 - acc: 0.6193 - val_loss: 0.6622 - val_acc: 0.6632
Epoch 2/5
2658/2658 [==============================] - 2s 615us/step - loss: 0.7654 - acc: 0.7863 - val_loss: 0.5824 - val_acc: 0.7526
Epoch 3/5
2658/2658 [==============================] - 2s 628us/step - loss: 0.6194 - acc: 0.8232 - val_loss: 0.6671 - val_acc: 0.7211
Epoch 4/5
2658/2658 [==============================] - 2s 630us/step - loss: 0.5373 - acc: 0.8582 - val_loss: 0.6327 - val_acc: 0.7526
Epoch 5/5
2658/2658 [==============================] - 2s 639us/step - loss: 0.4614 - acc: 0.8751 - val_loss: 0.6795 - val_acc: 0.7456
[[1791  292]
 [  18  557]]
0.7823033707865168
0.9686956521739131
[[694 219]
 [ 71 156]]
0.5182724252491695
0.6872246696035242


In [74]:
# User Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.User_Type, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 5.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 2658 samples, validate on 1140 samples
Epoch 1/5
2658/2658 [==============================] - 3s 1ms/step - loss: 0.4448 - acc: 0.9379 - val_loss: 0.2118 - val_acc: 0.9351
Epoch 2/5
2658/2658 [==============================] - 2s 586us/step - loss: 0.2340 - acc: 0.9579 - val_loss: 0.1836 - val_acc: 0.9351
Epoch 3/5
2658/2658 [==============================] - 2s 614us/step - loss: 0.1699 - acc: 0.9676 - val_loss: 0.1578 - val_acc: 0.9500
Epoch 4/5
2658/2658 [==============================] - 2s 596us/step - loss: 0.1346 - acc: 0.9763 - val_loss: 0.1479 - val_acc: 0.9640
Epoch 5/5
2658/2658 [==============================] - 2s 627us/step - loss: 0.1168 - acc: 0.9804 - val_loss: 0.1554 - val_acc: 0.9535
[[2533   34]
 [  13   78]]
0.768472906403941
0.8571428571428571
[[1060   28]
 [  25   27]]
0.5046728971962617
0.5192307692307693
